In [1]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

TRAIN_DIR = 'E:\Courses\youtube\DeepLearning_with_Tensorflow\CNN\Dogs vs. Cats Redux\train'
TEST_DIR = 'E:\Courses\youtube\DeepLearning_with_Tensorflow\CNN\Dogs vs. Cats Redux\test'
IMAGE_SIZE = 50
LR = 1e-3

MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '2conv-basic')

In [2]:
def label_img(img):
    word_label = img.split('.')[-3]
    return [1,0] if word_label=='cat' else [0,1] if word_label=='dog' else None    

In [3]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMAGE_SIZE, IMAGE_SIZE))
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data) 
    np.save('train_data.npy', training_data)
    return training_data    